# geolocation demo using ai
### you can contact me on discord @inputoutputcontrol

## installation & loading

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!pip install -q geoclip
from geoclip import GeoCLIP
import torch
if torch.cuda.is_available():
  model = GeoCLIP().to("cuda")
else:
  model = GeoCLIP()
print("loaded")
from PIL import Image
from io import BytesIO
from google.colab import files
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

## image upload & predictions

In [ ]:
uploadd = files.upload()
if uploadd:
    imagename = next(iter(uploadd))
    imagepath = BytesIO(uploadd[imagename])
    print("uploaded")
    image = Image.open(imagepath)
    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.axis('off')
    plt.show()
else:
    print("no image")
top_pred_gps, top_pred_prob = model.predict(imagepath, top_k=50)
print("predictions:")
for i in range(5):
    lat, lon = top_pred_gps[i]
    prob_percent = top_pred_prob[i] * 100
    print(f"{i+1}: ({lat:.6f}, {lon:.6f}) - probability: {prob_percent:.2f}%")

## mapping

In [ ]:
topncoordinates = 10
gpscoordinates = top_pred_gps.tolist()[:topncoordinates]
probabilities = top_pred_prob.tolist()[:topncoordinates]
total_prob = sum(probabilities)
normalizedprobs = [prob / total_prob for prob in probabilities]
weightedcoordinates = [(lat, lon, weight) for (lat, lon), weight in zip(gpscoordinates, normalizedprobs)]
avglat = sum(lat for lat, lon, weight in weightedcoordinates) / len(weightedcoordinates)
avglon = sum(lon for lat, lon, weight in weightedcoordinates) / len(weightedcoordinates)
m = folium.Map(location=[avglat, avglon], zoom_start=2.2)
magma = {
    0.0: '#932667',
    0.2: '#b5367a',
    0.4: '#d3466b',
    0.6: '#f1605d',
    0.8: '#fd9668',
    1.0: '#fcfdbf'
}
HeatMap(weightedcoordinates, gradient=magma).add_to(m)
topcoordinate = gpscoordinates[0]
topprobability = normalizedprobs[0]
folium.Marker(
    location=topcoordinate,
    popup=f"{topcoordinate};{topprobability:.4f}",
    icon=folium.Icon(color='orange', icon='star')
).add_to(m)
m